# Contador de palavras com PySpark

### O "Hello World" em Ciência de Dados

In [17]:
from pyspark.sql import SparkSession, functions as F
spark = (
    SparkSession
    .builder
    .appName('WordCount')
    .getOrCreate()
)

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords =stopwords.words('portuguese')

sc = spark.sparkContext


[nltk_data] Downloading package stopwords to /home/andre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
path_file = '/home/andre/Downloads/biblia-sagrada.txt'
words_bible = sc.textFile(path_file)

Número de linhas

In [19]:
# Quantidade de linhas do arquivo
print(words_bible.count())

32368


In [20]:
# Dez primeiras linhas do arquivo que ainda não sofreu alterações.
words_bible.take(10)

['########################################################################    ',
 '',
 '        BÍBLIA SAGRADA',
 '        - ANTIGO E VELHO TESTAMENTO',
 '',
 '######################################################################## ',
 '',
 'ANTIGO TESTAMENTO',
 '',
 'GÊNESIS']

In [21]:
'''
  Método para remover a caracteres especiais e 
  deixar todas as palavras em minúsculas
'''
def lower_clean_str(x_str):
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x_str.lower().strip()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

In [22]:
words_bible = words_bible.map(lower_clean_str)
words_bible.take(10)


['',
 '',
 'bíblia sagrada',
 ' antigo e velho testamento',
 '',
 '',
 '',
 'antigo testamento',
 '',
 'gênesis']

In [23]:
# Usando a função split() para deixar cada palavra em uma linha dentro de uma lista
words_bible = words_bible.flatMap(lambda line: line.split(" "))

In [24]:
words_bible.take(5)

['', '', 'bíblia', 'sagrada', '']

Excluindo linhas vazias

In [25]:
words_bible = words_bible.filter(lambda x:x !='')
words_bible.take(5)

['bíblia', 'sagrada', 'antigo', 'e', 'velho']

### Contando a frequência em que cada palavra aparece.

In [26]:
words_bible = words_bible.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)



In [27]:
print(words_bible.count())
words_bible.take(5)

29100


[('sagrada', 12), ('antigo', 11), ('testamento', 7), ('1', 1254), ('no', 3933)]

In [28]:
'''
    Caso a lista words_bible tenha alguma palavra
    exibida neste print abaixo, será removida, pois
    não faz sentido contá-las.
'''

#biblioteca importada mais acima
print(stopwords)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [29]:
'''
    Ignorando alguns pronomes e verbos que não
    agregam na contagem
'''
words_bible = words_bible.filter(lambda word: word[0] not in stopwords)

In [30]:
# Total de palavras após o tratamento
print(words_bible.count())

28909


### Criando um dataFrame Pyspark

Capturando as dez palavras mais mencionadas no novo e velho testamento da Bíblia

In [31]:
df_count_words = spark.createDataFrame(words_bible, 'Palavras STRING, Total INTEGER')

df_count_words = (
    df_count_words
    .withColumn('Palavras', F.when( F.col('Palavras') == 'deus', 'Deus'  )
        .otherwise(df_count_words.Palavras) 
    )
)

df_count_words.orderBy('Total', ascending = False).show(10)

+--------+-----+
|Palavras|Total|
+--------+-----+
|  senhor| 8016|
|    Deus| 4417|
|    pois| 3382|
|   terra| 2936|
|  filhos| 2781|
|  porque| 2774|
|   sobre| 2761|
|   então| 2547|
|  israel| 2542|
|     rei| 2475|
+--------+-----+
only showing top 10 rows

